In [44]:
import pandas as pd
import numpy as np
import json
from tqdm import tqdm

In [45]:
df = pd.read_csv('county_data.csv')
df.head()

,year,state_po,county_name,county_fips,totalvotes,dvotes,rvotes,statevotes,vote_percentage,r_margin_perc
0,2000,AL,AUTAUGA,1001.0,17208,4942,11993,1672551,0.010288,0.004216
1,2000,AL,BALDWIN,1003.0,56480,13997,40872,1672551,0.033769,0.016068
2,2000,AL,BARBOUR,1005.0,10395,5188,5096,1672551,0.006215,-0.000055
3,2000,AL,BIBB,1007.0,7101,2710,4273,1672551,0.004246,0.000935
4,2000,AL,BLOUNT,1009.0,17973,4977,12667,1672551,0.010746,0.004598


In [46]:
with open('counties-albers-10m.json', 'r') as file:
    topo = json.load(file)

In [47]:
counties = topo["objects"]["counties"]["geometries"]

In [48]:
fips = df["county_fips"].unique().astype(int).astype(str)
f = np.vectorize(lambda x : x.zfill(5))
fips = f(fips)
fips

C:\Users\nitzr\AppData\Local\Temp\ipykernel_40300\1775769219.py:1: RuntimeWarning: invalid value encountered in cast
  fips = df["county_fips"].unique().astype(int).astype(str)


array(['01001', '01003', '01005', ..., '02039', '02040', '02099'],
      dtype='<U11')

In [49]:
df[df["county_fips"] == 1001][["year", "totalvotes", "dvotes", "rvotes", "statevotes", "vote_percentage", "r_margin_perc"]]

,year,totalvotes,dvotes,rvotes,statevotes,vote_percentage,r_margin_perc
0,2000,17208,4942,11993,1672551,0.010288,0.004216
3116,2004,20081,4758,15196,1883415,0.010662,0.005542
6233,2008,23641,6093,17403,2099819,0.011259,0.005386
9350,2012,23932,6363,17379,2070353,0.011559,0.005321
12467,2016,24973,5936,18172,2123367,0.011761,0.005763
15789,2020,27770,7503,19838,2323282,0.011953,0.005309


In [50]:
for code in tqdm(fips):
    code_found = False
    for county in counties:
        if county["id"] == code:
            properties = county["properties"]
            properties["data"] = []
            county_df = df[df["county_fips"] == int(code)][["year", "totalvotes", "dvotes", "rvotes", "statevotes", "vote_percentage", "r_margin_perc"]]
            for row in county_df.itertuples():
                datapoint = {
                    "year":row.year,
                    "totalvotes":row.totalvotes,
                    "dvotes":row.dvotes,
                    "rvotes":row.rvotes,
                    "vote_perc":row.vote_percentage,
                    "r_marg":row.r_margin_perc
                }
                properties["data"].append(datapoint)
            county["properties"] = properties
            code_found = True
            break
        
topo["objects"]["counties"]["geometries"] = counties

100%|██████████| 3157/3157 [00:02<00:00, 1575.89it/s]


In [51]:
with open("../site/data/county_map.json", 'w', encoding='utf-8') as file:
    json.dump(topo, file, ensure_ascii=False, indent=4)